In [26]:
from constants import *
from data_loading import *
from data_cleaning import *
#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)


# load training set as DataFrame
train = load_df(TRAIN_NEG, TRAIN_POS, TWEET, LABEL, LABEL_NEG, LABEL_POS)
# set patterns to remove, replace, and replace with
to_remove = "<user>"
to_replace = "[^a-zA-Z#]"
replace_value = " "
# clean training set
train = clean(train, TWEET, CLEAN_TWEET, to_remove, to_replace, replace_value)

In [27]:
train.head()

,tweet,label,clean_tweet
id,,,
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,0,vinco tresorpack difficulti object disassembl ...
1,glad i dot have taks tomorrow ! ! #thankful #s...,0,glad dot tak tomorrow #thank #startho
2,1-3 vs celtics in the regular season = were fu...,0,vs celtic regular season fuck play playoff
3,<user> i could actually kill that girl i'm so ...,0,could actual kill girl sorri
4,<user> <user> <user> i find that very hard to ...,0,find hard believ im afraid


### Bag of Words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['clean_tweet'])

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_bow = bow

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['label'], random_state=42, test_size=0.3)


In [5]:
lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int) # calculating f1 score

0.7786465875084468

### TF IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(train['clean_tweet'])

In [7]:
train_tfidf = tfidf

xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

lreg.fit(xtrain_tfidf, ytrain)

prediction = lreg.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)

0.7774145944529492

### Fasttext (embedding)

In [28]:
from embeddings import *

#model = fasttext_model(train, model = "skipgram")
#print(model.words)
# TODO use embedding

### Fasttext (prediction)

#### on "clean_tweet"

In [29]:
create_labeled_csv(train, tweets_col = "clean_tweet")
classifier = fasttext.supervised('./data/fasttext_train.txt', 'model_supervised', label_prefix='__label__')

result = classifier.test('./data/fasttext_train.txt')
print('P@1:', result.precision)
print('R@1:', result.recall)
print('Number of examples:', result.nexamples)

P@1: 0.710955
R@1: 0.710955
Number of examples: 200000


#### on "clean_tweet"

In [30]:
create_labeled_csv(train, tweets_col = "tweet")
classifier = fasttext.supervised('./data/fasttext_train.txt', 'model_supervised', label_prefix='__label__')

result = classifier.test('./data/fasttext_train.txt')
print('P@1:', result.precision)
print('R@1:', result.recall)
print('Number of examples:', result.nexamples)

P@1: 0.841965
R@1: 0.841965
Number of examples: 200000


In [11]:
# TODO fix MemoryError 
#classifier2 = fasttext.load_model('models/wiki-news-300d-1M-subword.vec')

Exception: fastText: Cannot load models/wiki-news-300d-1M-subword.vec due to C++ extension failed to allocate the memory

In [ ]:
result = classifier.test('./data/fasttext_train.txt')
print('P@1:', result.precision)
print('R@1:', result.recall)
print('Number of examples:', result.nexamples)